In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from LSsurf import smooth_xytb_fit
import pointCollection as pc
import sparseqr
import glob
import h5py
import os
import LSsurf


In [10]:
%matplotlib widget

In [108]:
def safe_interp(x, x0_in, y0_in):
    y=np.NaN
    
    if x0_in[-1] < x0_in[0]:
        x0=x0_in[::-1]
        y0=y0_in[::-1]
    else:
        x0=x0_in
        y0=y0_in
    try:
        i0=np.argwhere(x0 < x)[-1][0]
        i1=np.argwhere(x0 >=x)[0][0]
        #print([i0, i1])
        #print( x0[[i0, i1]])
        #print( y0[[i0, i1]])
        y=np.interp(x, x0[[i0, i1]], y0[[i0, i1]])
    except Exception:
        pass
    return y


In [ ]:
! ls -lt /home/besmith4/shared/ATL11_processing/Arctic_003_cycle_03_09/003 | head

In [13]:
data_top='/home/besmith4/shared/ATL11_processing/Arctic_003_cycle_03_09/003'
#data_top='/Data/ATL11'
all_files =   glob.glob(data_top+'/ATL11*01.h5') 
sorted_files = sorted(all_files, key = os.path.getsize, reverse=True)

In [ ]:
len(all_files)
file=sorted_files[0]
print(file)
#D11=pc.ATL11.data().from_h5(file)


In [12]:
sorted_files=['/home/besmith4/shared/ATL11_processing/Arctic_003_cycle_03_09/003/ATL11_091703_0309_003_01.h5']

for file in sorted_files[0:1]:
    
    D11=pc.ATL11.data().from_h5(file)
   
    plt.figure(1, figsize=[6, 4]); plt.clf()
    plt.subplot(131)
    plt.plot(D11.x_atc, D11.h_corr,'.')
    plt.subplot(132)
    ii = (D11.fit_quality[:,0]==0).ravel()
    plt.plot(D11.x_atc[ii,:], D11.h_corr[ii,:],'.')
    plt.title(os.path.basename(file))
   
    plt.subplot(133)
    plt.plot(D11.x_atc, np.sum(np.isfinite(D11.h_corr), axis=1))
D11.index(D11.fit_quality[:,0] ==0)
   

<class 'pointCollection.ATL11.data.data'> with shape (35708, 7),
with fields:
['latitude', 'longitude', 'h_corr', 'h_corr_sigma', 'h_corr_sigma_systematic', 'delta_time', 'quality_summary', 'ref_pt', 'dem_h', 'x_atc', 'fit_quality', 'cycle_number']

In [4]:

# define the domain's width in x, y, and time
W={'x':4.e4,'y':400,'t':.2}
# define the grid center:
XR=np.nanmean(D11.x_atc)+np.array([-1, 1])*W['x']/2
ctr={'x':XR[0]+W['x']/2., 'y':0., 't':0.}
# define the grid spacing
spacing={'z0':100, 'dz':100, 'dt':.1}


In [5]:
D=pc.data().from_dict({'x':D11.x_atc[:,0], 'y':np.zeros_like(D11.x_atc[:,0]),'z':D11.h_corr[:,0],\
                       'time':np.zeros_like(D11.x_atc[:,0]), 'sigma':D11.h_corr_sigma[:,0]})
# To ensure a time-constant simulation, replicate the data at times -0.5 and 0.5:
#data=pc.data().from_list([D, D.copy().assign({'time':np.zeros_like(D.x)}), D.copy().assign({'time':np.zeros_like(D.x)+0.5})])
data=D
data.index(np.isfinite(data.z) & np.isfinite(data.sigma) & (data.sigma>0))


<class 'pointCollection.data.data'> with shape (33123,),
with fields:
['x', 'y', 'z', 'time', 'sigma']

In [ ]:
! gdalsrsinfo -o proj4 EPSG:3413

In [ ]:
2+2

In [7]:
# define the expected statistics of the surface
from LSsurf import smooth_xytb_fit
E_d3zdx2dt=0.0001
E_d2z0dx2=0.006
E_d2zdt2=5000

data_gap_scale=2500

data1=data[(data.x > XR[0]) & (data.x < XR[0]+W['x'])]
srs_proj4='+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs '
mask_file='/home/besmith4/nobackup/masks/Arctic/GimpIceMask_100m_edited.tif'
scale_vals=np.array([ 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 10, 100])[::-1]
sigma_hat_vals=np.zeros_like(scale_vals)
sigma_hat_s_vals=np.zeros_like(scale_vals)
N_vals=np.zeros_like(scale_vals)
S=[]
d_ed=[]
for ii, scale_val in enumerate(scale_vals):
    print(scale_val)
    # run the fit
    E_RMS={'d2z0_dx2': E_d2z0dx2*scale_val,
     'dz0_dx': E_d2z0dx2*data_gap_scale*scale_val,
     'd3z_dx2dt':E_d3zdx2dt ,
     'd2z_dxdt': E_d3zdx2dt*data_gap_scale,
     'd2z_dt2': E_d2zdt2}
    srs_proj4=None
    S.append(smooth_xytb_fit(data=data1, ctr=ctr, W=W, spacing=spacing, E_RMS=E_RMS,
                     reference_epoch=1, N_subset=None, compute_E=False,
                     max_iterations=5,
                     VERBOSE=False, dzdt_lags=[1]))
    d_ed.append(S[-1]['data'])
    d_ed[-1].index(d_ed[-1].three_sigma_edit==1)
    sigma_hat_vals[ii] = LSsurf.RDE(d_ed[-1].z-d_ed[-1].z_est)
    N_vals[ii]=d_ed[-1].size
    sigma_hat_s_vals[ii] = LSsurf.RDE((d_ed[-1].z-d_ed[-1].z_est)/d_ed[-1].sigma)


100.0
10.0
1.0
0.3
0.1
0.03
0.01
0.003
0.001


In [133]:
    
# plot the results
fig=plt.figure( figsize=[6,6])
fig.clf()
x0 = data1.x[0]

ax=[]
ax.append(fig.add_subplot(221))
ax[-1].plot( (data1.x-x0)/1000, data1.z,'ko', label='data', zorder=0)
ax[-1].plot((S[-1]['m']['z0'].x-x0)/1000, S[-1]['m']['z0'].z0[0,:],'r',linewidth=1.5, label='z0,tight constraint', zorder=2)
ax[-1].plot((S[0]['m']['z0'].x-x0)/1000, S[0]['m']['z0'].z0[0,:],'b', linewidth=1.5, label='z0, loose constraint', zorder=1)
ax[-1].set_ylabel('height, m')
ax[-1].set_xlabel('x_atc, km')
ax[-1].legend();

ax.append(fig.add_subplot(222, sharex=ax[0]))
ax[-1].plot((d_ed[-1].x-x0)/1000, (d_ed[-1].z-d_ed[-1].z_est),'r.', label='tight constraint')
ax[-1].plot((d_ed[0].x-x0)/1000, (d_ed[0].z-d_ed[0].z_est),'b.', label='loose constraint')
ax[-1].set_ylabel('residual, m')
ax[-1].set_xlabel('x_atc, km')
ax[-1].legend()

ax.append(fig.add_subplot(223, sharex=ax[0]))
ax[-1].plot((d_ed[-1].x-x0)/1000, (d_ed[-1].z-d_ed[-1].z_est)/d_ed[-1].sigma,'r.', label='tight constraint')
ax[-1].plot((d_ed[0].x-x0)/1000, (d_ed[0].z-d_ed[0].z_est)/d_ed[0].sigma,'b.', label='loose constraint')
ax[-1].set_ylabel('scaled residual')
ax[-1].set_xlabel('x_atc, km')

ax.append( fig.add_subplot(224))
ax[-1].loglog(scale_vals*E_d2z0dx2, sigma_hat_s_vals, label='robust scaled residual')
ax[-1].loglog(scale_vals*E_d2z0dx2, N_vals/S[0]['data'].size, label='fraction pts used')


this_x0=safe_interp( 1.,sigma_hat_s_vals, scale_vals*E_d2z0dx2)
yl=ax[-1].get_ylim()
ax[-1].plot(this_x0*np.ones(2), hax[1].get_ylim(), 'k--', linewidth=2, label='$\sigma_{xx}$'+f'={this_x0:2.2e}')
ax[-1].set_ylim(yl)

ax[-1].legend()
ax[-1].set_xlabel('$\sigma_{xx}$')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
fig.savefig('ATL11_z0_param_selection.tif', format='tif')

AttributeError: __delete__

0.00018278340762478408

In [ ]:
MOG=pc.grid.data().from_geotif('/Data/MOG/mog1km_2005_hp1_v1.1.tif');
MOG.show()
D1=D11[(D11.x_atc[:,0] > XR[0]) & (D11.x_atc[:,0] < XR[0]+W['x'])]
D1.get_xy(EPSG=3413)
plt.plot(D1.x, D1.y,'r.')

In [ ]:
np.max((scale_vals*E_d2z0dx2)[sigma_hat_s_vals>1])

In [ ]:
def read_ATL11_file(file, mask_file):

    D11=pc.ATL11.data().from_h5(file)
    with h5py.File(file,'r') as h5f:
        qs=np.array(h5f['/pt2/ref_surf/quality_summary'])
    D11.assign({'ref_surf_quality':qs})
    D11.get_xy(EPSG=3413)
    XR=np.array([np.nanmin(D11.x), np.nanmax(D11.x)])
    YR=np.array([np.nanmin(D11.y), np.nanmax(D11.y)])
    mask=pc.grid.data().from_geotif(mask_file, bounds=[XR, YR]).interp(D11.x[:,0], D11.y[:,0]) > 0.5
    D11.index(mask & (D11.ref_surf_quality <1))
    return D11

In [ ]:
def find_best_wxx0(D11):

    scale_vals=np.array([ 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300])

    E_d3zdx2dt=0.0001
    E_d2z0dx2=0.006
    E_d2zdt2=5000

    data_gap_scale=2500

    # define the domain's width in x, y, and time
    W={'x':4.e4,'y':200,'t':.2}
    # define the grid center:
    XR=np.nanmean(D11.x_atc)+np.array([-1, 1])*W['x']/2
    ctr={'x':XR[0]+W['x']/2., 'y':0., 't':0.}
    # define the grid spacing
    spacing={'z0':100, 'dz':100, 'dt':.1}

    dN=np.ceil(W['x']/30).astype(int)

    L_interp={}

    for pt0 in np.arange(D11.ref_pt[0,0]+dN/2, D11.ref_pt[-1,0], dN):
        ii=np.flatnonzero(np.abs(D11.ref_pt[:,0]-pt0)<3*dN/2)
        N_good=np.sum(np.isfinite(D11.h_corr[ii,:]), axis=0)
        if np.max(N_good)<0.9*dN:
            continue
        bc=np.argmax(N_good)
        nb=N_good[bc]

        xy_ctr=[np.nanmean(D11.x[ii, bc]), np.nanmean(D11.y[ii, bc]), np.nanmean(D11.h_corr[ii, bc])]

        D=pc.data().from_dict({'x':D11.x_atc[ii,bc], 'y':np.zeros_like(ii, dtype=float),'z':D11.h_corr[ii,bc],\
                           'time':np.zeros_like(ii, dtype=float), 'sigma':D11.h_corr_sigma[ii,bc]})
        D.index(np.isfinite(D.z) & np.isfinite(D.sigma) & (D.sigma>0))
        S=[]
        ctr={'x':np.nanmean(D.x), 'y':0., 't':0.}

        L_curve={key:[] for key in ['wzz0', 'sigma_hat_s', 'N']}

        for  scale_val in scale_vals:
            # run the fit
            E_RMS={'d2z0_dx2': E_d2z0dx2*scale_val,
             'dz0_dx': E_d2z0dx2*data_gap_scale*scale_val,
             'd3z_dx2dt':E_d3zdx2dt ,
             'd2z_dxdt': E_d3zdx2dt*data_gap_scale,
             'd2z_dt2': E_d2zdt2}
            S.append(smooth_xytb_fit(data=D, ctr=ctr, W=W, spacing=spacing, E_RMS=E_RMS,
                         reference_epoch=1, N_subset=None, compute_E=False,
                         max_iterations=5,
                         VERBOSE=False, dzdt_lags=[1]))
            d_ed = S[-1]['data']
            d_ed.index(d_ed.three_sigma_edit==1)

            L_curve['sigma_hat_s'].append( LSsurf.RDE((d_ed.z-d_ed.z_est)/d_ed.sigma))
            L_curve['wzz0'].append(E_RMS['d2z0_dx2'])
            L_curve['N'].append(d_ed.size)
        for key in L_curve.keys():
            L_curve[key] = np.array(L_curve[key])

        L_interp[pt0] = {"w_for_r_of_1":safe_interp(1, L_curve['sigma_hat_s'], L_curve['wzz0']), 
                        'w_for_r_10pct_above_min':safe_interp(1.1*L_curve['sigma_hat_s'].min(), L_curve['sigma_hat_s'], L_curve['wzz0']), 
                        'x': xy_ctr[0], 
                        'y': xy_ctr[1],
                        'z': xy_ctr[2]}
    return L_interp

In [ ]:
! ls '/home/besmith4/nobackup/masks'

In [ ]:

D11=read_ATL11_file(sorted_files[10],'/home/besmith4/nobackup/masks/Arctic/GimpIceMask_100m_edited.tif' )
L_interp=find_best_wxx0(D11)

fig=plt.figure(6); plt.clf()
h0=fig.add_subplot(211)
plt.plot(D11.ref_pt[:,0], D11.h_corr,'.')
fig.add_subplot(212, sharex=h0)
plt.plot(np.array(list(L_interp.keys())), np.log10(np.array([L_interp[key]['w_for_r_10_pct_above_min'] for key in L_interp.keys()])),'ks')
plt.plot(np.array(list(L_interp.keys())), np.log10(np.array([L_interp[key]['w_for_r_of_1'] for key in L_interp.keys()])),'r*')



## Make a queue of files to analyze

In [32]:
themask=pc.grid.data().from_geotif('/home/besmith4/nobackup/masks/Arctic/GimpIceMask_100m_edited.tif')[::20, ::20]
readme=np.zeros(len(sorted_files), dtype=bool)
in_list=[]
out_list=[]
fail_list=[]
for file_count, file in enumerate(sorted_files):
    try:
        with h5py.File(file,'r') as h5f:
            Dll=pc.data().from_dict({'latitude':np.array(h5f['/pt2/latitude'])[::10], 'longitude':np.array(h5f['/pt2/longitude'])[::10]})
        Dll.get_xy(EPSG=3413)
    except Exception as e:
        fail_list += [file]
        continue
    N_good=np.nansum(themask.interp(Dll.x, Dll.y)>0.5)
    if N_good > 50:
        in_list += [file]
    else:
        out_list += [file]
    

In [46]:
import re
re_11=re.compile('\d\d_01.h5')
! mkdir /home/besmith4/nobackup/ATL11_wxx_analysis_v1/
mask_file='/home/besmith4/nobackup/masks/Arctic/GimpIceMask_100m_edited.tif'
with open('/home/besmith4/temp/wxx_queue.txt','w') as fh:
    for file in in_list:
        if re_11.search(file) is None:
            continue
        out_file = '/home/besmith4/nobackup/ATL11_wxx_analysis_v1/'+os.path.basename(file)
        fh.write(f'source activate IS2; python3 ~/git_repos/surfaceChange/scripts//best_wxx0_for_ATL11.py {file} {mask_file} {out_file}\n')

mkdir: cannot create directory ‘/home/besmith4/nobackup/ATL11_wxx_analysis_v1/’: File exists


In [47]:
out_files=glob.glob('/home/besmith4/nobackup/ATL11_wxx_analysis_v1/*.h5')
D_list=[]
for file in out_files:
    D_list.append(pc.data().from_h5(file, field_dict={None:['w_for_r_10pct_above_min','w_for_r_of_1', 'x','y','z' ]}))

w_for_r_10pct_above_min  Dataset {13/Inf}
w_for_r_of_1             Dataset {13/Inf}
x                        Dataset {13/Inf}
y                        Dataset {13/Inf}
z                        Dataset {13/Inf}

In [48]:
D_all=pc.data().from_list(D_list)
D_all.index(np.isfinite(D_all.w_for_r_of_1))

<class 'pointCollection.data.data'> with shape (12681,),
with fields:
['w_for_r_10pct_above_min', 'w_for_r_of_1', 'z', 'y', 'x']

In [49]:
plt.figure(figsize=[8,4]);
plt.clf()
hax=plt.gcf().subplots(1,2)

ii=np.argsort(D_all.w_for_r_of_1)
plt.sca(hax[0])
plt.scatter(D_all.x[ii], D_all.y[ii], 4, c=np.log10(D_all.w_for_r_of_1[ii]), vmin=-4, vmax=-1.5); 
plt.axis('equal')
plt.colorbar( shrink=0.5, extend='both', orientation='horizontal', label='$log_{10} (\sigma_{xx0})$');

hax[1].hist(np.log10(D_all.w_for_r_of_1), np.arange(-4.5, -1.5, 0.1))
hax[1].set_xlabel('$log_{10} (\sigma_{xx0})$')
hax[1].yaxis.set_ticks_position('right')
hax[1].plot(np.log10(0.006)*np.ones(2), hax[1].get_ylim(),'k--')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
ux0=[]
nx0=[]
D_mb0=[]
for dxi in [-1.e4, 0, 1.e4]:
    for dyi in [-1.e4, 0, 1.e4]:
        x0=np.round((D_all.x+1j*D_all.y-(dxi+1j*dyi))/2.e4)*2.e4+(dxi+1j*dyi)
        ux0i=np.unique(x0)
        nx0i=np.zeros_like(ux0i, dtype=float)
        D_mb0i=np.zeros_like(ux0i, dtype=float)
        for ii, xx0 in enumerate(ux0i):
            these=x0==xx0
            nx0i[ii]=np.sum(these)
            D_mb0i[ii]=np.nanmedian(D_all.w_for_r_of_1[these])
        ux0 += [ux0i]
        nx0 += [nx0i]
        D_mb0 += [D_mb0i]

ux0=np.concatenate(ux0)
nx0=np.concatenate(nx0)
D_mb0=np.concatenate(D_mb0)

ux0=ux0[nx0>3]
D_mb0=D_mb0[nx0>3]
nx0=nx0[nx0>3]


In [77]:
_, temp=np.unique(ux0, return_index=True)
ux0=ux0[temp]
D_mb0=D_mb0[temp]
nx0=nx0[temp]

In [121]:
plt.figure( figsize=[5,3.5]);
plt.clf()
hax=plt.gcf().subplots(1,2)

ii=np.argsort(D_mb0)
plt.sca(hax[0])
plt.scatter(np.real(ux0[ii]), np.imag(ux0[ii]), 6, c=np.log10(D_mb0[ii]), vmin=-4, vmax=-2.5)
plt.colorbar( shrink=0.5, extend='both',  label='$log_{10} (\sigma_{xx})$');
plt.gca().set_xticks([])
plt.gca().set_yticks([])
#orientation='horizontal',

hax[1].hist(np.log10(D_mb0), np.arange(-4.1, -2.5, 0.025))
hax[1].set_xlabel('$log_{10} (\sigma_{xx})$')
#hax[1].yaxis.set_ticks_position('right')
#hax[1].yaxis.tick_right()
this_p98=scoreatpercentile(D_mb0, 98)
hax[1].plot(np.log10(this_p98)*np.ones(2), hax[1].get_ylim(),'k--', label="$\sigma_{xx}$="+f"{this_p98:2.1e}")
hax[1].set_ylabel('count')
hax[1].legend()
plt.tight_layout()

plt.axis('equal');  
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [126]:
plt.gcf().savefig('sigma_xx_map.png', format='png')

AttributeError: __delete__

In [ ]:
plt.gcf()

In [107]:
pwd

'/att/gpfsfs/home/besmith4/git_repos/surfaceChange/notebooks'

In [70]:
from scipy.stats import scoreatpercentile


In [124]:
this_p98/np.sqrt(8)

0.0005864714439832331